In [2]:
import numpy as np
import pickle
import os, re, json

In [7]:
classes = [x[1] for x in os.walk('../data/review/KDD2014-Chen-Dataset/Non-Electronics')][0]

In [62]:
classes = ['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
class_uri = ['/c/en/atheism',
            '/c/en/graphics',
            '/c/en/operating_system',
            '/c/en/ibm',
            '/c/en/mac',
            '/c/en/windows',
            '/c/en/sale',
            '/c/en/auto',
            '/c/en/motorcycle',
            '/c/en/baseball',
            '/c/en/hockey',
            '/c/en/crypt',
            '/c/en/electronics',
            '/c/en/medical',
            '/c/en/space',
            '/c/en/christian',
            '/c/en/gun',
            '/c/en/middle_east',
            '/c/en/politics',
            '/c/en/religion']

In [15]:
PATHS_FROM = pickle.load(open("../wordEmbeddings/PATHS_FROM_01.pickle", "rb"))

AttributeError: Can't get attribute 'Path' on <module '__main__'>

In [14]:
print(2 < '2')

TypeError: '<' not supported between instances of 'int' and 'str'

In [2]:
KG_VECTORS_2 = pickle.load(open("../wordEmbeddings/KG_VECTORS_2.pickle", "rb"))

In [3]:
for key in KG_VECTORS_2:
    print(key, len(KG_VECTORS_2[key]))

/c/en/company 21267
/c/en/education 7514
/c/en/artist 7035
/c/en/athlete 5156
/c/en/officer 12797
/c/en/transport 14151
/c/en/building 34099
/c/en/nature 24775
/c/en/village 11549
/c/en/animal 62420
/c/en/plant 46970
/c/en/album 5381
/c/en/film 15207
/c/en/writing 18824


In [2]:
data = np.load('../results/test_5.npz')

In [3]:
print(data.keys())

['align_unseen', 'gt_seen', 'pred_unseen', 'align_seen', 'pred_seen', 'gt_unseen']


In [5]:
print(data['gt_unseen'], data['pred_unseen'], data['align_unseen'])

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] [[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]] [[[2.47972156e-03 2.73659639e-03 2.02324498e-03 ... 3.87769453e-02
   9.77866575e-02 9.69534069e-02]
  [9.36909858e-03 9.62136686e-03 7.42427306e-03 ... 2.30694991e-02
   2.38265380e-01 3.20462227e-01]
  [2.78055595e-05 3.19405553e-05 2.46378622e-05 ... 4.07097396e-04
   5.02543598e-02 9.46804345e-01]
  ...
  [1.32638263e-03 1.42502296e-03 1.00284070e-03 ... 2.21102890e-02
   7.81787410e-02 7.99784884e-02]
  [5.52700879e-03 5.93346125e-03 4.26079705e-03 ... 2.06842422e-02
   1.01126134e-01 1.16921619e-01]
  [6.46062428e-04 6.23823900e-04 4.90807986e-04 ... 2.92846859e-02
   1.86704919e-01 2.34603584e-01]]

 [[6.90063986e-04 9.84093989e-04 1.23169867e-03 ... 3.82701159e-02
   3.81

In [66]:
def process_text(filename):
    f = open(filename, 'r')
    lines = [line for line in f]
    f.close()
    
    processed = []
    for line in lines:
        if re.search(r'[a-zA-Z0-9]', line) is None:
            line = ''
        line = re.sub(r'([\w.-]+)@([\w.-]+)', 'EMAIL', line)
        line = re.sub(r'[^A-Za-z0-9_.\-\"\',()?\s]', ' ', line)
#         line = re.sub(r'\s+', ' ', line)
#         print(line)
        processed.append(line)
    ans = ''.join(processed)
    ans = re.sub(r'\s+', ' ', ans)
    return ans

In [68]:
print(process_text('..\\data\\newsgroup\\20news-18828\\talk.politics.misc\\176865'))

From EMAIL (Steve Hendricks) Subject Re Price Controls for Medical Care (WAS Re We're from the govt...) In article EMAIL EMAIL writes In article EMAIL , EMAIL (Steve Hendricks) writes Thanks to Kim for following up. I was hoping that someone would bring up the issue of cost compensation. The problem with the argument is that it fails to explain why kidney dialysis (RD) services have expanded massively in the last decade. After all, no one is forcing private providers to offer this service. If they are losing money on the treatment (which according to information I've collected from several providers they are not) why would they not simply limit their losses by cutting back on services (engage in effective rationing of supply) rather than expand the coverage to a larger market that must then be compensated by raising prices in other areas? Perhaps there is a competitive advantage in there after all. Would not reputable and dedicated physicians prefer to affiliate with an institution tha

In [67]:
for c in classes:
    directory = '..\\data\\newsgroup\\20news-18828-preprocessed\\'+c
    if not os.path.exists(directory):
        os.makedirs(directory)
    for root, dirs, files in os.walk('..\\data\\newsgroup\\20news-18828\\'+c):
        for f in files:
            text = process_text(os.path.join(root, f))
            with open(os.path.join(directory, f), 'w') as the_file:
                the_file.write(text)

In [4]:
data = json.load(open('..\\data\\review\\50EleReviews.json', 'r'))

In [10]:
print(len(data['X']), data['target_names'])

50000 ['Monitor', 'Watch', 'Blu Ray Player', 'CDPlayer', 'Network Adapter', 'Battery', 'Graphics Card', 'Xbox', 'Telephone', 'Car Stereo', 'Video Recorder', 'Alarm Clock', 'Camera', 'Camcorder', 'HardDrive', 'Home Theater System', 'Subwoofer', 'Rice Cooker', 'Webcam', 'Kindle', 'Scanner', 'Radar Detector', 'Vacuum', 'Cable Modem', 'CellPhone', 'Wireless Router', 'Printer', 'Fan', 'Amplifier', 'Microwave', 'GPS', 'Microphone', 'TV', 'Projector', 'Mouse', 'Voice Recorder', 'Memory Card', 'Iron', 'Laptop', 'Headphone', 'Video Player', 'MP3 Player', 'Keyboard', 'Tablet', 'Media Player', 'Lamp', 'Remote Control', 'Speaker', 'Computer', 'DVDPlayer']
